In [1]:
from pathlib import Path
import csv
import numpy as np
import cv2
import sklearn
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Flatten,MaxPooling2D,Dropout
from keras.layers.convolutional import Convolution2D
import matplotlib.image as mpimg
from keras import backend as K
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
data = []
with open('./Subset.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        data.append([row[0],int(row[-1])])
print(data[1])

['PlantVillage-Dataset/raw/color/Grape___Black_rot/9eb19238-5388-4e56-88a0-8351da034735___FAM_B.Rot 0545.JPG', 0]


In [3]:
train_samples, validation_samples = train_test_split(data, test_size=0.2)

In [4]:
def preprocess(image_list):
    not_match = []
    images=[]
    labels=[]
    for image_lable in image_list:                  
        image = mpimg.imread(image_lable[0])
        image = cv2.resize(image, (196, 196))
        images.append(image)
        labels.append(image_lable[-1])
#     print(images)
#     print(lables)
    return images,labels

In [5]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1:
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_images,batch_lables= preprocess(samples[offset:offset+batch_size])
            x_train=np.asfarray(batch_images)
            #x_train=keras.utils.to_categorical(x_train, num_classes=32)
            y_train=np.asfarray(batch_lables)
            y_train=keras.utils.to_categorical(y_train, num_classes=38)
            yield x_train, y_train

In [6]:
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [ ]:
model = Sequential()
model.add(Convolution2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(196,196,3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(38, activation='softmax'))
#model.compile(loss='mse',optimizer='adam')
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=2)
model.save('model-1.h5')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_steps=1480, validation_data=<generator..., steps_per_epoch=5920, epochs=2)`


Epoch 1/2
 129/5920 [..............................] - ETA: 9:13:47 - loss: 15.6845 - acc: 0.0269